# AIVLE스쿨 4기 DX트랙 5차 미니프로젝트 
## [미션#1] API를 이용하여 공공 데이터 가져오기

 
 - <font size =+1> 공공데이터포털(https://data.go.kr)에서 제공되는 [국립중앙의료원 - 전국응급의료기관 조회 서비스]에서 필요한 자료를 API를 통해 가져옵니다.
 - <font size =+1> 회원 가입 후 해당 자료에 [활용 신청]을 한 뒤, 마이페이지에서 일반 인증키(encoding)을 확인합니다. (PDF 교재 참고)
 - <font size =+1> 대구광역시 중심으로 데이터 추출 
 

 * 사전 확인 사항 
   * haversine 설치
   * python : 3.11.3 (아나콘다 2023.07 버전 설치 기준)
 * 잘 진행하다가 중간에 [ValueError: xpath does not return any nodes]가 발생되면 API 서버 문제일 수 있음. 잠시 후 다시 실행해 보기

In [733]:
import pandas as pd # 필요한 라이브러리 불러오기
import numpy as np
from urllib.parse import quote

In [734]:
import ssl # 코드 수정 없이 실행
from urllib.request import urlopen
context=ssl.create_default_context()
context.set_ciphers("DEFAULT")

* 공공데이터 수집을 위한 기본값 설정

In [735]:
# 개인 인증키 저장
key = "AsSDRMS8YsGyYCeyYZPWC%2BN6QSRKweQpU27iKc0HZeeSzPRlmk7jFERIjPJzH3%2FFwQc0UqtoPv1VYEXCiXFckA%3D%3D" # [국립중앙의료원 - 전국응급의료기관 조회 서비스] 활용을 위한 개인 일반 인증키(Encoding) 값 저장

# 조회할 도시 지정
city = quote("대구광역시") # city = 대구광역시, 인코딩 필요

### 1) 대구광역시 응급의료기관 목록정보 조회하기

 * 주소 (대구광역시)를 기준으로 응급 의료 기관 목록 정보 조회
 * 기관ID, 주소, 기관명, 응급실전화, 병원 위치(경도/위도) 정보를 확인

In [736]:
# 대구광역시내이 응급의료기관 목록정보 조회하기
# url : https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytListInfoInqire
      
# 인증키 지정 : serviceKey = key  ## 갱
# 도시 지정 : Q0 = city
# 출력 행수 : numOfRows = 100 
# page 번호 : pageNo = 1

url = "https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytListInfoInqire?serviceKey=" +key + "&Q0=" + city + "&numOfRows=100&pageNo=1"
# url = "http://apis.data.go.kr/B552657/ErmctInfoInqireService/getEmrrmRltmUsefulSckbdInfoInqire?serviceKey=" +key + "&Q0=" + city + "&numOfRows=100&pageNo=1"

# url = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytListInfoInqire?serviceKey=' + key + '&Q0=' + city+ '&numOfRows=100&pageNo=1'
print(f"url : \n{url}")
result = urlopen(url, context = context)
emrList = pd.read_xml(result, xpath = '//item')
emrList.head(5)

url : 
https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytListInfoInqire?serviceKey=AsSDRMS8YsGyYCeyYZPWC%2BN6QSRKweQpU27iKc0HZeeSzPRlmk7jFERIjPJzH3%2FFwQc0UqtoPv1VYEXCiXFckA%3D%3D&Q0=%EB%8C%80%EA%B5%AC%EA%B4%91%EC%97%AD%EC%8B%9C&numOfRows=100&pageNo=1


dutyAddr dutyEmcls dutyEmclsName  \
0  대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)      G007      지역응급의료기관   
1        대구광역시 동구 동촌로 207, 강남병원 (방촌동)      G007      지역응급의료기관   
2   대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)      G001      권역응급의료센터   
3  대구광역시 중구 달성로 56, 계명대학교대구동산병원 (동산동)      G007      지역응급의료기관   
4          대구광역시 달서구 달구벌대로 1035 (신당동)      G006      지역응급의료센터   

                dutyName      dutyTel1      dutyTel3      hpid     phpid  \
0  (재)미리내천주성삼성직수도회천주성삼병원  053-790-1000  053-790-0555  A1300081  A1300081   
1                   강남병원  053-980-9000  053-980-9019  A1300018  A1300018   
2                경북대학교병원  053-200-5114  053-200-5100  A1300002  A1300002   
3            계명대학교대구동산병원  053-250-8114  053-250-8481  A1303386  A1303386   
4              계명대학교동산병원     1577-6622  053-258-6301  A1300003  A1300003   

   rnum   wgs84Lat    wgs84Lon  
0     1  35.840225  128.705733  
1     2  35.883009  128.663456  
2     3  35.866235  128.604315  
3     4  35.869460  128.582931  
4     5  35.853887  128.480130

In [737]:
emrList.shape # 응급의료기관 수 확인하기 : shape 

(21, 11)

In [738]:
emrList.columns

Index(['dutyAddr', 'dutyEmcls', 'dutyEmclsName', 'dutyName', 'dutyTel1',
       'dutyTel3', 'hpid', 'phpid', 'rnum', 'wgs84Lat', 'wgs84Lon'],
      dtype='object')

In [739]:
## 응급의료기관 목록정보 중 기관코드(hpid), 응급기관명(dutyName), 주소(dutyAddr), 응급실연락처(dutyTel3), 위도(wgs84Lat), 경도(wgs84Lon) 정보만 추출
## solution_df 에 저장하기
solution_df = emrList[['hpid', 'dutyName', 'dutyAddr', 'dutyTel3', 'wgs84Lat', 'wgs84Lon']].copy() ## ermList 중 [hpid, dutyName, dutyAddr, dutyTel3, wgs84Lat, wgs84Lon] 컬럼 활용
solution_df.head()

hpid               dutyName                            dutyAddr  \
0  A1300081  (재)미리내천주성삼성직수도회천주성삼병원  대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)   
1  A1300018                   강남병원        대구광역시 동구 동촌로 207, 강남병원 (방촌동)   
2  A1300002                경북대학교병원   대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)   
3  A1303386            계명대학교대구동산병원  대구광역시 중구 달성로 56, 계명대학교대구동산병원 (동산동)   
4  A1300003              계명대학교동산병원          대구광역시 달서구 달구벌대로 1035 (신당동)   

       dutyTel3   wgs84Lat    wgs84Lon  
0  053-790-0555  35.840225  128.705733  
1  053-980-9019  35.883009  128.663456  
2  053-200-5100  35.866235  128.604315  
3  053-250-8481  35.869460  128.582931  
4  053-258-6301  35.853887  128.480130

### 2) 응급의료기관별 기본정보 조회 - 응급실수, 수술실 수 확인

 * <b>기관ID</b>를 기준으로 응급실, 수술실 정보 등 응급의료기관 기본 정보 조회 가능
 * 응급실수 (hperyn)와 수술실 수(hpopyn) 정보를 확인

In [740]:
# 대구광역시내이 응급의료기관 목록정보 조회하기
# url : https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytBassInfoInqire
# 인증키 지정 : serviceKey = key
# 응급의료기관 지정 : HPID = hpid
# 응급의료기관 하나씩 조회해야 함

# 응급의료기관에 대한 hpid값을 list로 추출
hpidList = list(solution_df['hpid']) 

# 각 응급의료기관의 응급실 병상수(hperyn), 수술실 수(hpopyn) 를 저장하기 위한 빈 리스트 생성
hperynList = [] 
hpopynList = [] 

# 대구광역시의 응급의료기관을 hpid 기준으로 하나씩 조회하여, 응급실 병상수/수술실 수를 확인 
# for문을 이용하여 hpid 하나씩 조회하고, hperyn, hpopyn 정보를 각 hperynList, hpopynList 리스트에 저장 
for hpid in hpidList: # 
    url_basic = f'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytBassInfoInqire?serviceKey={key}&HPID={hpid}' 
    result = urlopen(url_basic, context=context) 
    df_temp = pd.read_xml(result, xpath='.//item') 
    # print(df_temp[['hperyn', 'hpopyn']]) 
    
    if 'hperyn' in list(df_temp.columns) : 
        hperynList.append(df_temp['hperyn'][0]) 
        
    else: 
        hperynList.append(0) 
    
    if 'hpopyn' in list(df_temp.columns) : 
        hpopynList.append(df_temp['hpopyn'][0]) 
    else: 
        hpopynList.append(0) 
hperynList

[10,
 10,
 22,
 0,
 28,
 10,
 11,
 24,
 10,
 7,
 10,
 28,
 10,
 10,
 10,
 30,
 11,
 0,
 17,
 14,
 0]

In [741]:
# 기존 solution_df에 응급실수, 수술실 수 정보 추가하기
# 응급실 수 : solution_df['hperyn']
# 수술실 수 : solution_df['hpopyn']

solution_df['hperyn'] = hperynList 
solution_df['hpopyn'] = hpopynList
# 확인하기
solution_df.head()

hpid               dutyName                            dutyAddr  \
0  A1300081  (재)미리내천주성삼성직수도회천주성삼병원  대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)   
1  A1300018                   강남병원        대구광역시 동구 동촌로 207, 강남병원 (방촌동)   
2  A1300002                경북대학교병원   대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)   
3  A1303386            계명대학교대구동산병원  대구광역시 중구 달성로 56, 계명대학교대구동산병원 (동산동)   
4  A1300003              계명대학교동산병원          대구광역시 달서구 달구벌대로 1035 (신당동)   

       dutyTel3   wgs84Lat    wgs84Lon  hperyn  hpopyn  
0  053-790-0555  35.840225  128.705733      10       4  
1  053-980-9019  35.883009  128.663456      10       4  
2  053-200-5100  35.866235  128.604315      22      16  
3  053-250-8481  35.869460  128.582931       0       0  
4  053-258-6301  35.853887  128.480130      28      24

In [742]:
## hperyn, hpopyn값이 0 이하인 병원 삭제
solution_df = solution_df[solution_df['hpopyn']>0] 
solution_df = solution_df[solution_df['hperyn']>0]

In [743]:
solution_df.shape ## solution_df 크기 확인 : shape

(18, 8)

In [744]:
## solution_df 를 파일로 저장 
solution_df.to_csv('daegu_hospital_list.csv', index = False) ## 파일명 : daegu_hospital_list.csv, index = False

In [745]:
solution_df = pd.read_csv('./daegu_hospital_list.csv') ## 정상적으로 저장되었는지 확인 
solution_df.head()

hpid               dutyName                            dutyAddr  \
0  A1300081  (재)미리내천주성삼성직수도회천주성삼병원  대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)   
1  A1300018                   강남병원        대구광역시 동구 동촌로 207, 강남병원 (방촌동)   
2  A1300002                경북대학교병원   대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)   
3  A1300003              계명대학교동산병원          대구광역시 달서구 달구벌대로 1035 (신당동)   
4  A1300008                    곽병원             대구광역시 중구 국채보상로 531 (수동)   

       dutyTel3   wgs84Lat    wgs84Lon  hperyn  hpopyn  
0  053-790-0555  35.840225  128.705733      10       4  
1  053-980-9019  35.883009  128.663456      10       4  
2  053-200-5100  35.866235  128.604315      22      16  
3  053-258-6301  35.853887  128.480130      28      24  
4  053-605-3333  35.870693  128.588728      10       4

### 3) 응급의료기관별 실시간 가용 병상 조회

 * 주소(대구광역시)를 기준으로 응급실 실시간 가용병상정보 등을 조회
 * 실시간 응급실(hvec), 수술실(hvoc) 가용 수량 정보 확인

In [746]:
# 응급실 실시간 가용병상 조회
# url : https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEmrrmRltmUsefulSckbdInfoInqire
# 인증키 지정 : serviceKey = key
# 도시 지정 : STAGE1=city
# 출력 행수 : numOfRows=100 
# page 번호 : pageNo = 1
# emrRealtime_big 변수명으로 저장

city = quote("대구광역시") # city = 대구광역시, 인코딩 필요

url_realtime = "https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEmrrmRltmUsefulSckbdInfoInqire?serviceKey=" + key + "&STAGE1=" + city + "&numOfRows=100&pageNo=1"

result = urlopen(url_realtime, context=context)
emrRealtime_big = pd.read_xml(result, xpath='//item')

emrRealtime_big.head()

dutyName      dutyTel3      hpid  hv27  hv30 hv5   hv7 hvamyn  \
0        드림종합병원  053-640-8575  A1300034   1.0   1.0   Y     Y      Y   
1    대구가톨릭대학교병원  053-650-3025  A1300004   NaN   NaN   Y     Y      Y   
2       대구굿모닝병원  053-620-9119  A1300076   NaN   1.0   Y     Y      Y   
3       영남대학교병원  053-620-3177  A1300001   0.0   0.0   Y     Y      Y   
4  의료법인구의료재단구병원  053-557-7119  A1300006   NaN   1.0   Y  None      Y   

  hvangioayn hvcrrtayn  ... hv15  hv16 hv33  hv37 hvs10  hvs20 hvs48 hvs49  \
0          Y        N1  ...  NaN   NaN  NaN   NaN   NaN    NaN   NaN   NaN   
1          Y         Y  ...  NaN   NaN  NaN   NaN   NaN    NaN   NaN   NaN   
2          Y        N1  ...  NaN   NaN  NaN   NaN   NaN    NaN   NaN   NaN   
3          Y         Y  ...  NaN   NaN  NaN   NaN   NaN    NaN   NaN   NaN   
4         N1        N1  ...  NaN   NaN  NaN   NaN   NaN    NaN   NaN   NaN   

   hv39 hvs23  
0   NaN   NaN  
1   NaN   NaN  
2   NaN   NaN  
3   NaN   NaN  
4   NaN   NaN  

[5 rows x 113 columns]

In [747]:
## 응급실 실시간 가용병상 정보에서 기관코드(hpid), 응급실 병상수('hvec'), 수술실 수('hvoc') 정보만 추출하여 emRealtime_small 변수에 저장
emrRealtime_small = emrRealtime_big[['hpid', 'hvec', 'hvoc']].copy() ## emrRealtime_big 중 [hpid, hvec, hvoc] 컬럼 활용
emrRealtime_small.head()

hpid  hvec  hvoc
0  A1300034     7     3
1  A1300004     4    17
2  A1300076     8     1
3  A1300001    10    26
4  A1300006     3     7

In [748]:
# solution_df와 emrRealtime_small 데이터프레임을 결합하여 solution_df에 저장
# 결합 : merge 함수 활용

solution_df = pd.merge(solution_df, emrRealtime_small, on = 'hpid',how='inner')
solution_df.head()

hpid               dutyName                            dutyAddr  \
0  A1300081  (재)미리내천주성삼성직수도회천주성삼병원  대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)   
1  A1300018                   강남병원        대구광역시 동구 동촌로 207, 강남병원 (방촌동)   
2  A1300002                경북대학교병원   대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)   
3  A1300003              계명대학교동산병원          대구광역시 달서구 달구벌대로 1035 (신당동)   
4  A1300008                    곽병원             대구광역시 중구 국채보상로 531 (수동)   

       dutyTel3   wgs84Lat    wgs84Lon  hperyn  hpopyn  hvec  hvoc  
0  053-790-0555  35.840225  128.705733      10       4    10     4  
1  053-980-9019  35.883009  128.663456      10       4    12     7  
2  053-200-5100  35.866235  128.604315      22      16   -17    16  
3  053-258-6301  35.853887  128.480130      28      24    -4    22  
4  053-605-3333  35.870693  128.588728      10       4     0     4

In [749]:
solution_df.shape

(18, 10)

### 4) 응급의료기관별 중증질환자 수용 가능 여부 조회

 * 주소(대구광역시)를 기준으로 중증질환자 수용 가능 여부를 조회 (실시간 정보)

In [750]:
# 실시간 중증질환자 수용 가능 병원 조회
# url : https://apis.data.go.kr/B552657/ErmctInfoInqireService/getSrsillDissAceptncPosblInfoInqire
# 인증키 지정 : serviceKey = key
# 도시 지정 : STAGE1=city
# 출력 행수 : numOfRows=100 
# page 번호 : pageNo = 1

url_acpt = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getSrsillDissAceptncPosblInfoInqire?serviceKey=' + key + "&STAGE1=" +city + "&numOfRows=100&pageNo=1"

result = urlopen(url_acpt, context=context)
emrAcpt_big = pd.read_xml(result, xpath='.//item')

## 다른 API함수와 다르게 기관코드 컬럼명이 다름 (hpid --> dutyName)
## 기관코드 컬렴명을 'hpid'로 일치화시키기 위해, 컬럼명을 변경함

# emrAcpt_big = 
emrAcpt_big.rename(columns = {'dutyName' : 'hpid'}, inplace = True)

emrAcpt_big.head()

hpid MKioskTy1 MKioskTy10 MKioskTy11 MKioskTy12 MKioskTy13 MKioskTy14  \
0  A1300034     정보미제공          Y          Y      정보미제공      정보미제공      정보미제공   
1  A1300004         Y      정보미제공          Y      정보미제공          Y      정보미제공   
2  A1300076     정보미제공      정보미제공      정보미제공      정보미제공      정보미제공      정보미제공   
3  A1300001         Y      정보미제공          Y      정보미제공          Y      정보미제공   
4  A1300006     정보미제공      정보미제공      정보미제공      정보미제공      정보미제공      정보미제공   

  MKioskTy15 MKioskTy16 MKioskTy17  ... MKioskTy5 MKioskTy6 MKioskTy7  \
0      정보미제공      정보미제공      정보미제공  ...     정보미제공     정보미제공         Y   
1          Y          Y          Y  ...         Y         Y         Y   
2      정보미제공      정보미제공      정보미제공  ...     정보미제공     정보미제공     정보미제공   
3          Y          Y          Y  ...         Y         Y     정보미제공   
4      정보미제공      정보미제공      정보미제공  ...     정보미제공     정보미제공         Y   

  MKioskTy8 MKioskTy9    MKioskTy10Msg MKioskTy12Msg MKioskTy14Msg  \
0         Y         Y             None          None          None   
1         Y         Y  21일 이상 출생아부터 가능       0세부터 가능             .   
2     정보미제공     정보미제공             None          None          None   
3         Y     정보미제공         전체 연령 가능     평일 외래만 가능          None   
4     정보미제공         Y             None          None          None   

  MKioskTy15Msg MKioskTy27Msg  
0          None          None  
1             .             .  
2          None          None  
3     전체 체중아 가능      전체 연령 가능  
4          None          None  

[5 rows x 34 columns]

In [751]:
## 실시간 중증질환자 수용 가능 병원정보에서 필요한 정보만 추출하여 emrAcpt_small 변수에 저장
## emrAcpt 중 [hpid, MKioskTy1, MKioskTy2, MKioskTy3, MKioskTy4, MKioskTy5, MKioskTy7,MKioskTy8, MKioskTy10, MKioskTy11] 컬럼 확인
# 'MKioskTy1' : '뇌출혈수술', 
# 'MKioskTy2' : '뇌경색의재관류',
# 'MKioskTy3' : '심근경색의재관류',
# 'MKioskTy4' : '복부손상의수술',
# 'MKioskTy5' : '사지접합의수술',
# 'MKioskTy7' : '응급투석',
# 'MKioskTy8' : '조산산모',
# 'MKioskTy10' : '신생아',
# 'MKioskTy11' : '중증화상'

emrAcpt_small = emrAcpt_big[['hpid', 'MKioskTy1', 'MKioskTy2', 'MKioskTy3', 'MKioskTy4', 'MKioskTy5', 'MKioskTy7','MKioskTy8', 'MKioskTy10', 'MKioskTy11']]
emrAcpt_small.head()

hpid MKioskTy1 MKioskTy2 MKioskTy3 MKioskTy4 MKioskTy5 MKioskTy7  \
0  A1300034     정보미제공     정보미제공     정보미제공     정보미제공     정보미제공         Y   
1  A1300004         Y         Y         Y         Y         Y         Y   
2  A1300076     정보미제공         Y         Y         Y     정보미제공     정보미제공   
3  A1300001         Y         Y       불가능       불가능         Y     정보미제공   
4  A1300006     정보미제공     정보미제공     정보미제공     정보미제공     정보미제공         Y   

  MKioskTy8 MKioskTy10 MKioskTy11  
0         Y          Y          Y  
1         Y      정보미제공          Y  
2     정보미제공      정보미제공      정보미제공  
3         Y      정보미제공          Y  
4     정보미제공      정보미제공      정보미제공

In [752]:
# solution_df와 emrAcpt_small 데이터프레임을 결합하여 solution_df에 저장
# 결합 : merge 함수 활용

solution_df = pd.merge(solution_df, emrAcpt_small, on = 'hpid',how = 'inner')
solution_df.head()

hpid               dutyName                            dutyAddr  \
0  A1300081  (재)미리내천주성삼성직수도회천주성삼병원  대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)   
1  A1300018                   강남병원        대구광역시 동구 동촌로 207, 강남병원 (방촌동)   
2  A1300002                경북대학교병원   대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)   
3  A1300003              계명대학교동산병원          대구광역시 달서구 달구벌대로 1035 (신당동)   
4  A1300008                    곽병원             대구광역시 중구 국채보상로 531 (수동)   

       dutyTel3   wgs84Lat    wgs84Lon  hperyn  hpopyn  hvec  hvoc MKioskTy1  \
0  053-790-0555  35.840225  128.705733      10       4    10     4     정보미제공   
1  053-980-9019  35.883009  128.663456      10       4    12     7     정보미제공   
2  053-200-5100  35.866235  128.604315      22      16   -17    16         Y   
3  053-258-6301  35.853887  128.480130      28      24    -4    22         Y   
4  053-605-3333  35.870693  128.588728      10       4     0     4     정보미제공   

  MKioskTy2 MKioskTy3 MKioskTy4 MKioskTy5 MKioskTy7 MKioskTy8 MKioskTy10  \
0     정보미제공     정보미제공     정보미제공     정보미제공         Y     정보미제공      정보미제공   
1     정보미제공     정보미제공     정보미제공     정보미제공     정보미제공     정보미제공      정보미제공   
2         Y         Y         Y         Y         Y         Y          Y   
3         Y         Y         Y         Y         Y     정보미제공          Y   
4     정보미제공     정보미제공     정보미제공     정보미제공         Y         Y      정보미제공   

  MKioskTy11  
0          Y  
1      정보미제공  
2          Y  
3          Y  
4          Y

### 5) 자료 정비하기

 * 컬럼명 한글로 변경하기
 * 데이터 중 '정보 미제공', '불가능'은 'N'로 변경
 * 응급실/수술실 수가 0 이하의 수치가 나오는 비정상 사례 존재 --> 0이하의 수치는 모두 0으로 대체하여 정비
 * 응급실 가용율 (응급실 가용 수량 / 응급실수)을 계산
 * 응급실 포화도는 응급실 가용율에 따라 [불가/혼잡/보통/원활] 4단계로 구분

In [753]:
## solution_df의 컬럼명 확인
solution_df.columns

Index(['hpid', 'dutyName', 'dutyAddr', 'dutyTel3', 'wgs84Lat', 'wgs84Lon',
       'hperyn', 'hpopyn', 'hvec', 'hvoc', 'MKioskTy1', 'MKioskTy2',
       'MKioskTy3', 'MKioskTy4', 'MKioskTy5', 'MKioskTy7', 'MKioskTy8',
       'MKioskTy10', 'MKioskTy11'],
      dtype='object')

In [754]:
# column명을 한글로 교체하기 위해 딕셔너리로 정의

column_change = {  'hpid' : '병원코드',
                   'dutyName' : '병원명',
                   'dutyAddr' : '주소',
                   'dutyTel3' : '응급연락처',
                   'wgs84Lat' : '위도', 
                   'wgs84Lon' : '경도',
                   'hperyn' : '응급실수',
                   'hpopyn' : '수술실수',
                   'hvec' : '가용응급실수',
                   'hvoc' : '가용수술실수',
                   'MKioskTy1' : '뇌출혈', 
                   'MKioskTy2' : '뇌경색',
                   'MKioskTy3' : '심근경색',
                   'MKioskTy4' : '복부손상',
                   'MKioskTy5' : '사지접합',
                   'MKioskTy7' : '응급투석',
                   'MKioskTy8' : '조산산모',
                   'MKioskTy10' : '신생아',
                   'MKioskTy11' : '중증화상',
                   
                   }

In [755]:
# solution_df의 column명 변경하기 : rename

solution_df = solution_df.rename(columns = column_change)
solution_df.head()

병원코드                    병원명                                  주소  \
0  A1300081  (재)미리내천주성삼성직수도회천주성삼병원  대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)   
1  A1300018                   강남병원        대구광역시 동구 동촌로 207, 강남병원 (방촌동)   
2  A1300002                경북대학교병원   대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)   
3  A1300003              계명대학교동산병원          대구광역시 달서구 달구벌대로 1035 (신당동)   
4  A1300008                    곽병원             대구광역시 중구 국채보상로 531 (수동)   

          응급연락처         위도          경도  응급실수  수술실수  가용응급실수  가용수술실수    뇌출혈  \
0  053-790-0555  35.840225  128.705733    10     4      10       4  정보미제공   
1  053-980-9019  35.883009  128.663456    10     4      12       7  정보미제공   
2  053-200-5100  35.866235  128.604315    22    16     -17      16      Y   
3  053-258-6301  35.853887  128.480130    28    24      -4      22      Y   
4  053-605-3333  35.870693  128.588728    10     4       0       4  정보미제공   

     뇌경색   심근경색   복부손상   사지접합   응급투석   조산산모    신생아   중증화상  
0  정보미제공  정보미제공  정보미제공  정보미제공      Y  정보미제공  정보미제공      Y  
1  정보미제공  정보미제공  정보미제공  정보미제공  정보미제공  정보미제공  정보미제공  정보미제공  
2      Y      Y      Y      Y      Y      Y      Y      Y  
3      Y      Y      Y      Y      Y  정보미제공      Y      Y  
4  정보미제공  정보미제공  정보미제공  정보미제공      Y      Y  정보미제공      Y

In [756]:
# 중증질환 수용 가능 여부 데이터 중 정보 미제공, 불가능은 N로 변경 : replace

solution_df = solution_df.replace("정보미제공","N")
solution_df = solution_df.replace("불가능","N")
solution_df.head()

병원코드                    병원명                                  주소  \
0  A1300081  (재)미리내천주성삼성직수도회천주성삼병원  대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)   
1  A1300018                   강남병원        대구광역시 동구 동촌로 207, 강남병원 (방촌동)   
2  A1300002                경북대학교병원   대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)   
3  A1300003              계명대학교동산병원          대구광역시 달서구 달구벌대로 1035 (신당동)   
4  A1300008                    곽병원             대구광역시 중구 국채보상로 531 (수동)   

          응급연락처         위도          경도  응급실수  수술실수  가용응급실수  가용수술실수 뇌출혈 뇌경색  \
0  053-790-0555  35.840225  128.705733    10     4      10       4   N   N   
1  053-980-9019  35.883009  128.663456    10     4      12       7   N   N   
2  053-200-5100  35.866235  128.604315    22    16     -17      16   Y   Y   
3  053-258-6301  35.853887  128.480130    28    24      -4      22   Y   Y   
4  053-605-3333  35.870693  128.588728    10     4       0       4   N   N   

  심근경색 복부손상 사지접합 응급투석 조산산모 신생아 중증화상  
0    N    N    N    Y    N   N    Y  
1    N    N    N    N    N   N    N  
2    Y    Y    Y    Y    Y   Y    Y  
3    Y    Y    Y    Y    N   Y    Y  
4    N    N    N    Y    Y   N    Y

In [757]:
## 응급실수/가용응급실수, 수술실수/가용수술실 수가 0보다 작은 경우는 비정상 데이터로 추정
## 0보다 작은 수는 0으로 변경
## loc 활용

solution_df.loc[solution_df['응급실수']<0, '응급실수'] = 0
solution_df.loc[solution_df['수술실수']<0, '수술실수'] = 0
solution_df.loc[solution_df['가용응급실수']<0, '가용응급실수'] = 0
solution_df.loc[solution_df['가용수술실수']<0, '가용수술실수'] = 0

In [758]:
# 응급실 가용율을 구하여 새로운 컬럼으로 추가하기
# 컬렴명 : '응급실가용율'
# 산식 : 가용 응급실수 / 응급실 수
# 소수 둘째 자리까지 구하기 round() 활용
# 응급실가용율 계산

solution_df['응급실가용율'] = round(solution_df['가용응급실수']/ solution_df['응급실수'],2)

# 응급실가용율 값이 1보다 큰 경우, 1로 설정
solution_df.loc[solution_df['응급실가용율'] > 1, '응급실가용율'] = 1

# 결과 확인
solution_df.head()

병원코드                    병원명                                  주소  \
0  A1300081  (재)미리내천주성삼성직수도회천주성삼병원  대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)   
1  A1300018                   강남병원        대구광역시 동구 동촌로 207, 강남병원 (방촌동)   
2  A1300002                경북대학교병원   대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)   
3  A1300003              계명대학교동산병원          대구광역시 달서구 달구벌대로 1035 (신당동)   
4  A1300008                    곽병원             대구광역시 중구 국채보상로 531 (수동)   

          응급연락처         위도          경도  응급실수  수술실수  가용응급실수  가용수술실수 뇌출혈 뇌경색  \
0  053-790-0555  35.840225  128.705733    10     4      10       4   N   N   
1  053-980-9019  35.883009  128.663456    10     4      12       7   N   N   
2  053-200-5100  35.866235  128.604315    22    16       0      16   Y   Y   
3  053-258-6301  35.853887  128.480130    28    24       0      22   Y   Y   
4  053-605-3333  35.870693  128.588728    10     4       0       4   N   N   

  심근경색 복부손상 사지접합 응급투석 조산산모 신생아 중증화상  응급실가용율  
0    N    N    N    Y    N   N    Y     1.0  
1    N    N    N    N    N   N    N     1.0  
2    Y    Y    Y    Y    Y   Y    Y     0.0  
3    Y    Y    Y    Y    N   Y    Y     0.0  
4    N    N    N    Y    Y   N    Y     0.0

In [759]:
# 응급실 가용율에 따라 포화도 분류
# 응급실 가용율 구분 단계 : ~0.1, 0.1 ~ 0.3, 0.3 ~ 0.6, 0.6 ~ 
# 포화도 명칭 : ['불가', '혼잡', '보통', '원활']
# pd.cut() 활용

bins = [-np.inf, 0.1, 0.3, 0.6, np.inf]

# 해당 구분에 따른 포화도 명칭
labels = ['불가', '혼잡', '보통', '원활']

solution_df['응급실포화도'] = pd.cut(solution_df['응급실가용율'], bins=bins, labels=labels)
solution_df.head()

병원코드                    병원명                                  주소  \
0  A1300081  (재)미리내천주성삼성직수도회천주성삼병원  대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)   
1  A1300018                   강남병원        대구광역시 동구 동촌로 207, 강남병원 (방촌동)   
2  A1300002                경북대학교병원   대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)   
3  A1300003              계명대학교동산병원          대구광역시 달서구 달구벌대로 1035 (신당동)   
4  A1300008                    곽병원             대구광역시 중구 국채보상로 531 (수동)   

          응급연락처         위도          경도  응급실수  수술실수  가용응급실수  가용수술실수  ... 뇌경색  \
0  053-790-0555  35.840225  128.705733    10     4      10       4  ...   N   
1  053-980-9019  35.883009  128.663456    10     4      12       7  ...   N   
2  053-200-5100  35.866235  128.604315    22    16       0      16  ...   Y   
3  053-258-6301  35.853887  128.480130    28    24       0      22  ...   Y   
4  053-605-3333  35.870693  128.588728    10     4       0       4  ...   N   

  심근경색 복부손상 사지접합 응급투석 조산산모 신생아 중증화상 응급실가용율  응급실포화도  
0    N    N    N    Y    N   N    Y    1.0      원활  
1    N    N    N    N    N   N    N    1.0      원활  
2    Y    Y    Y    Y    Y   Y    Y    0.0      불가  
3    Y    Y    Y    Y    N   Y    Y    0.0      불가  
4    N    N    N    Y    Y   N    Y    0.0      불가  

[5 rows x 21 columns]

### 6) 상황에 따른 병원 데이터 출력하기

 * 상황1) 응급실 수용이 가능한 병원
 * 상황2) 중증질환자 수용이 가능하고 응급실 수용이 가능한 병원

In [760]:
# 상황1) 응급실 수용이 가능한 병원
# 조건1 : 응급실 포화도가 불가가 아닌 병원을 출력

condition1 = ( solution_df['응급실포화도'] != '불가')
solution_df[condition1].head()

병원코드                    병원명                                  주소  \
0  A1300081  (재)미리내천주성삼성직수도회천주성삼병원  대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)   
1  A1300018                   강남병원        대구광역시 동구 동촌로 207, 강남병원 (방촌동)   
6  A1300004             대구가톨릭대학교병원          대구광역시 남구 두류공원로17길 33 (대명동)   
8  A1300076                대구굿모닝병원              대구광역시 남구 성당로 224 (대명동)   
9  A1300007                  대구의료원              대구광역시 서구 평리로 157 (중리동)   

          응급연락처         위도          경도  응급실수  수술실수  가용응급실수  가용수술실수  ... 뇌경색  \
0  053-790-0555  35.840225  128.705733    10     4      10       4  ...   N   
1  053-980-9019  35.883009  128.663456    10     4      12       7  ...   N   
6  053-650-3025  35.843066  128.567426    24    18       4      17  ...   Y   
8  053-620-9119  35.852967  128.574067     7     2       8       1  ...   Y   
9  053-560-7288  35.859623  128.540561    10     4      16       4  ...   N   

  심근경색 복부손상 사지접합 응급투석 조산산모 신생아 중증화상 응급실가용율  응급실포화도  
0    N    N    N    Y    N   N    Y   1.00      원활  
1    N    N    N    N    N   N    N   1.00      원활  
6    Y    Y    Y    Y    Y   N    Y   0.17      혼잡  
8    Y    Y    N    N    N   N    N   1.00      원활  
9    N    N    N    N    N   N    Y   1.00      원활  

[5 rows x 21 columns]

In [761]:
# 상황2) 중증질환자 수용이 가능하고, 응급실 수용이 가능한 병원
# 조건1 : '뇌출혈' 중증질환자 수용 및 수술실 수용 가능한(가용 수술실수가 1이상) 응급의료기관 조회
# 조건2 : 응급실 포화도가 불가가 아닌 병원을 출력

special_m = '뇌출혈'
condition1 = (solution_df[special_m] == 'Y' ) & ( solution_df['가용수술실수']>=1 )

condition2 = (solution_df['응급실포화도']!='불가')

solution_df[condition1 & condition2]


병원코드         병원명                                                 주소  \
6   A1300004  대구가톨릭대학교병원                         대구광역시 남구 두류공원로17길 33 (대명동)   
14  A1300001     영남대학교병원                             대구광역시 남구 현충로 170 (대명동)   
16  A1300010   칠곡경북대학교병원  대구광역시 북구 호국로 807, 칠곡경북대학교병원(임상실습동 1~11층 포함) (학정동)   

           응급연락처         위도          경도  응급실수  수술실수  가용응급실수  가용수술실수  ... 뇌경색  \
6   053-650-3025  35.843066  128.567426    24    18       4      17  ...   Y   
14  053-620-3177  35.847506  128.585131    30    19      10      26  ...   Y   
16  053-200-2100  35.956083  128.564408    17    13       6      19  ...   Y   

   심근경색 복부손상 사지접합 응급투석 조산산모 신생아 중증화상 응급실가용율  응급실포화도  
6     Y    Y    Y    Y    Y   N    Y   0.17      혼잡  
14    N    N    Y    N    Y   N    Y   0.33      보통  
16    Y    Y    N    N    Y   N    Y   0.35      보통  

[3 rows x 21 columns]

### 7) 병원 거리 구하기

 * 병원 위치(위도/경도)와 환자의 위치(임의의 위도/경도) 간의 거리 측정
 * haversine 라이브러리를 활용 (https://pypi.org/project/haversine/)
 * 병원 ~ 환자간의 거리를 ['2km이내', '5km이내', '10km이내', '10km이상'] 4단계로 구분

In [762]:
# 환자와 병원간의 거리 측정을 위위한 라이브러리 설치
# !pip install haversine 

In [763]:
from haversine import haversine # 라이브러리 import

# 거리 측정 예시
location1 = (35.209710, 129.004751) # 위치1 : (35.209710, 129.004751)
location2 = (35.1631139, 129.1635509) # 위치2 : (35.1631139, 129.1635509)

print(round( haversine(location1, location2, unit = 'km'), 2) )

15.33


In [764]:
#Latitude, Longitude # 환자 위치 : 서문시장 내
patient= (35.8690760, 128.5811616)  

# 환자와 병원간 거리를 저장할 빈 리스트 만들기  # 변수명 : distance
distance = [] 

# solution_df의 복사본을 만들어 distance_df에 저장하기
distance_df = solution_df.copy() 

for idx, row in distance_df.iterrows():
    # haversine 함수를 이용하여 환자와 병원간 거리 구하기 
    # distance 변수에 하나씩 채우기
    # print(row['위도'], row['경도'] )
    distance.append( round( haversine( patient, ( row['위도'], row['경도']) , unit = 'km'), 2) ) # 단위 : km # 소수 둘째자리까지 구하기

distance_df['거리'] = distance #distance_df의 '거리' 컬럼에, distance 값 저장하기
distance_df #확인하기

병원코드                    병원명  \
0   A1300081  (재)미리내천주성삼성직수도회천주성삼병원   
1   A1300018                   강남병원   
2   A1300002                경북대학교병원   
3   A1300003              계명대학교동산병원   
4   A1300008                    곽병원   
5   A1300038                나사렛종합병원   
6   A1300004             대구가톨릭대학교병원   
7   A1300005        대구가톨릭대학교칠곡가톨릭병원   
8   A1300076                대구굿모닝병원   
9   A1300007                  대구의료원   
10  A1300009                대구파티마병원   
11  A1300110                  더블유병원   
12  A1300034                 드림종합병원   
13  A1300045                   삼일병원   
14  A1300001                영남대학교병원   
15  A1300006           의료법인구의료재단구병원   
16  A1300010              칠곡경북대학교병원   
17  A1300011       한국보훈복지의료공단대구보훈병원   

                                                   주소         응급연락처  \
0                  대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)  053-790-0555   
1                        대구광역시 동구 동촌로 207, 강남병원 (방촌동)  053-980-9019   
2                   대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)  053-200-5100   
3                          대구광역시 달서구 달구벌대로 1035 (신당동)  053-258-6301   
4                             대구광역시 중구 국채보상로 531 (수동)  053-605-3333   
5                       대구광역시 달서구 월배로 97, 나사렛병원 (진천동)  053-643-3119   
6                          대구광역시 남구 두류공원로17길 33 (대명동)  053-650-3025   
7                           대구광역시 북구 칠곡중앙대로 440 (읍내동)  053-320-2111   
8                              대구광역시 남구 성당로 224 (대명동)  053-620-9119   
9                              대구광역시 서구 평리로 157 (중리동)  053-560-7288   
10                              대구광역시 동구 아양로 99 (신암동)  053-940-7119   
11                         대구광역시 달서구 달구벌대로 1632 (감삼동)  053-550-5019   
12                             대구광역시 남구 대명로 153 (대명동)  053-640-8575   
13               대구광역시 달서구 월배로 436, 지하1,지상1~11층 (송현동)  053-659-3119   
14                             대구광역시 남구 현충로 170 (대명동)  053-620-3177   
15                           대구광역시 달서구 감삼북길 141 (감삼동)  053-557-7119   
16  대구광역시 북구 호국로 807, 칠곡경북대학교병원(임상실습동 1~11층 포함) (학정동)  053-200-2100   
17                             대구광역시 달서구 월곡로 60 (도원동)  053-630-7014   

           위도          경도  응급실수  수술실수  가용응급실수  가용수술실수  ... 심근경색 복부손상 사지접합  \
0   35.840225  128.705733    10     4      10       4  ...    N    N    N   
1   35.883009  128.663456    10     4      12       7  ...    N    N    N   
2   35.866235  128.604315    22    16       0      16  ...    Y    Y    Y   
3   35.853887  128.480130    28    24       0      22  ...    Y    Y    Y   
4   35.870693  128.588728    10     4       0       4  ...    N    N    N   
5   35.814700  128.525095    11     5       0       5  ...    Y    Y    N   
6   35.843066  128.567426    24    18       4      17  ...    Y    Y    Y   
7   35.934880  128.549252    10     3       0       3  ...    N    N    N   
8   35.852967  128.574067     7     2       8       1  ...    Y    Y    N   
9   35.859623  128.540561    10     4      16       4  ...    N    N    N   
10  35.883934  128.623835    28    13       0      11  ...    Y    Y    N   
11  35.852642  128.544969    10     8       8       7  ...    N    N    N   
12  35.839635  128.574355    10     3       7       3  ...    N    N    N   
13  35.832748  128.553855    10     2       7       4  ...    N    N    N   
14  35.847506  128.585131    30    19      10      26  ...    N    N    Y   
15  35.850823  128.541483    11     8       3       7  ...    N    N    N   
16  35.956083  128.564408    17    13       6      19  ...    Y    Y    N   
17  35.803137  128.551646    14     6       9       6  ...    N    N    N   

   응급투석 조산산모 신생아 중증화상 응급실가용율 응급실포화도     거리  
0     Y    N   N    Y   1.00     원활  11.68  
1     N    N   N    N   1.00     원활   7.57  
2     Y    Y   Y    Y   0.00     불가   2.11  
3     Y    N   Y    Y   0.00     불가   9.26  
4     Y    Y   N    Y   0.00     불가   0.71  
5     Y    Y   N    Y   0.00     불가   7.88  
6     Y    Y   N    Y   0.17     혼잡   3.15  
7     N    N   N    Y   0.00     불가   7.86  
8     N    N   N    N   1.00     원활   1.90  
9 

In [765]:
# distancd_df의 '거리'에 따라 거리구분 분류 : cut()
# '거리구분' 명칭 : ['2km이내', '5km이내', '10km이내', '10km이상']

bins = [-1, 2, 5, 10, np.inf ] 
labels = ['2km이내', '5km이내', '10km이내', '10km이상']

distance_df['거리구분'] = pd.cut(distance_df['거리'], bins = bins, labels = labels )
distance_df.head()

병원코드                    병원명                                  주소  \
0  A1300081  (재)미리내천주성삼성직수도회천주성삼병원  대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)   
1  A1300018                   강남병원        대구광역시 동구 동촌로 207, 강남병원 (방촌동)   
2  A1300002                경북대학교병원   대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)   
3  A1300003              계명대학교동산병원          대구광역시 달서구 달구벌대로 1035 (신당동)   
4  A1300008                    곽병원             대구광역시 중구 국채보상로 531 (수동)   

          응급연락처         위도          경도  응급실수  수술실수  가용응급실수  가용수술실수  ... 복부손상  \
0  053-790-0555  35.840225  128.705733    10     4      10       4  ...    N   
1  053-980-9019  35.883009  128.663456    10     4      12       7  ...    N   
2  053-200-5100  35.866235  128.604315    22    16       0      16  ...    Y   
3  053-258-6301  35.853887  128.480130    28    24       0      22  ...    Y   
4  053-605-3333  35.870693  128.588728    10     4       0       4  ...    N   

  사지접합 응급투석 조산산모 신생아 중증화상 응급실가용율 응급실포화도     거리    거리구분  
0    N    Y    N   N    Y    1.0     원활  11.68  10km이상  
1    N    N    N   N    N    1.0     원활   7.57  10km이내  
2    Y    Y    Y   Y    Y    0.0     불가   2.11   5km이내  
3    Y    Y    N   Y    Y    0.0     불가   9.26  10km이내  
4    N    Y    Y   N    Y    0.0     불가   0.71   2km이내  

[5 rows x 23 columns]

In [766]:
# 가까운 거리순으로 : '거리구분
# 응급실이 원활한 순서로 : '응급실포화도'
# 정렬하기 : sort_values

distance_df.sort_values(['거리구분', '응급실포화도'], ascending = [True, False])

병원코드                    병원명  \
8   A1300076                대구굿모닝병원   
4   A1300008                    곽병원   
9   A1300007                  대구의료원   
11  A1300110                  더블유병원   
12  A1300034                 드림종합병원   
13  A1300045                   삼일병원   
14  A1300001                영남대학교병원   
6   A1300004             대구가톨릭대학교병원   
15  A1300006           의료법인구의료재단구병원   
2   A1300002                경북대학교병원   
10  A1300009                대구파티마병원   
1   A1300018                   강남병원   
17  A1300011       한국보훈복지의료공단대구보훈병원   
16  A1300010              칠곡경북대학교병원   
3   A1300003              계명대학교동산병원   
5   A1300038                나사렛종합병원   
7   A1300005        대구가톨릭대학교칠곡가톨릭병원   
0   A1300081  (재)미리내천주성삼성직수도회천주성삼병원   

                                                   주소         응급연락처  \
8                              대구광역시 남구 성당로 224 (대명동)  053-620-9119   
4                             대구광역시 중구 국채보상로 531 (수동)  053-605-3333   
9                              대구광역시 서구 평리로 157 (중리동)  053-560-7288   
11                         대구광역시 달서구 달구벌대로 1632 (감삼동)  053-550-5019   
12                             대구광역시 남구 대명로 153 (대명동)  053-640-8575   
13               대구광역시 달서구 월배로 436, 지하1,지상1~11층 (송현동)  053-659-3119   
14                             대구광역시 남구 현충로 170 (대명동)  053-620-3177   
6                          대구광역시 남구 두류공원로17길 33 (대명동)  053-650-3025   
15                           대구광역시 달서구 감삼북길 141 (감삼동)  053-557-7119   
2                   대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)  053-200-5100   
10                              대구광역시 동구 아양로 99 (신암동)  053-940-7119   
1                        대구광역시 동구 동촌로 207, 강남병원 (방촌동)  053-980-9019   
17                             대구광역시 달서구 월곡로 60 (도원동)  053-630-7014   
16  대구광역시 북구 호국로 807, 칠곡경북대학교병원(임상실습동 1~11층 포함) (학정동)  053-200-2100   
3                          대구광역시 달서구 달구벌대로 1035 (신당동)  053-258-6301   
5                       대구광역시 달서구 월배로 97, 나사렛병원 (진천동)  053-643-3119   
7                           대구광역시 북구 칠곡중앙대로 440 (읍내동)  053-320-2111   
0                  대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)  053-790-0555   

           위도          경도  응급실수  수술실수  가용응급실수  가용수술실수  ... 복부손상 사지접합 응급투석  \
8   35.852967  128.574067     7     2       8       1  ...    Y    N    N   
4   35.870693  128.588728    10     4       0       4  ...    N    N    Y   
9   35.859623  128.540561    10     4      16       4  ...    N    N    N   
11  35.852642  128.544969    10     8       8       7  ...    N    N    N   
12  35.839635  128.574355    10     3       7       3  ...    N    N    Y   
13  35.832748  128.553855    10     2       7       4  ...    N    N    Y   
14  35.847506  128.585131    30    19      10      26  ...    N    Y    N   
6   35.843066  128.567426    24    18       4      17  ...    Y    Y    Y   
15  35.850823  128.541483    11     8       3       7  ...    N    N    Y   
2   35.866235  128.604315    22    16       0      16  ...    Y    Y    Y   
10  35.883934  128.623835    28    13       0      11  ...    Y    N    Y   
1   35.883009  128.663456    10     4      12       7  ...    N    N    N   
17  35.803137  128.551646    14     6       9       6  ...    N    N    N   
16  35.956083  128.564408    17    13       6      19  ...    Y    N    N   
3   35.853887  128.480130    28    24       0      22  ...    Y    Y    Y   
5   35.814700  128.525095    11     5       0       5  ...    Y    N    Y   
7   35.934880  128.549252    10     3       0       3  ...    N    N    N   
0   35.840225  128.705733    10     4      10       4  ...    N    N    Y   

   조산산모 신생아 중증화상 응급실가용율 응급실포화도     거리    거리구분  
8     N   N    N   1.00     원활   1.90   2km이내  
4     Y   N    Y   0.00     불가   0.71   2km이내  
9     N   N    Y   1.00     원활   3.81   5km이내  
11    N   N    N   0.80     원활   3.74   5km이내  
12    Y   Y    Y   0.70     원활   3.33   5km이내  
13    Y   Y    Y   0.70     원활   4.73   5km이내  
14    Y   N    Y   0.33     보통   2.43   5km이내  
6     Y   N    Y   0.17     혼잡   3.15   5km이내  
15    N   N    N   0

### 8) 중증 질환자를 수용 가능한 응급의료기관을 가까운 거리순으로 출력

In [767]:
distance_df

병원코드                    병원명  \
0   A1300081  (재)미리내천주성삼성직수도회천주성삼병원   
1   A1300018                   강남병원   
2   A1300002                경북대학교병원   
3   A1300003              계명대학교동산병원   
4   A1300008                    곽병원   
5   A1300038                나사렛종합병원   
6   A1300004             대구가톨릭대학교병원   
7   A1300005        대구가톨릭대학교칠곡가톨릭병원   
8   A1300076                대구굿모닝병원   
9   A1300007                  대구의료원   
10  A1300009                대구파티마병원   
11  A1300110                  더블유병원   
12  A1300034                 드림종합병원   
13  A1300045                   삼일병원   
14  A1300001                영남대학교병원   
15  A1300006           의료법인구의료재단구병원   
16  A1300010              칠곡경북대학교병원   
17  A1300011       한국보훈복지의료공단대구보훈병원   

                                                   주소         응급연락처  \
0                  대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)  053-790-0555   
1                        대구광역시 동구 동촌로 207, 강남병원 (방촌동)  053-980-9019   
2                   대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)  053-200-5100   
3                          대구광역시 달서구 달구벌대로 1035 (신당동)  053-258-6301   
4                             대구광역시 중구 국채보상로 531 (수동)  053-605-3333   
5                       대구광역시 달서구 월배로 97, 나사렛병원 (진천동)  053-643-3119   
6                          대구광역시 남구 두류공원로17길 33 (대명동)  053-650-3025   
7                           대구광역시 북구 칠곡중앙대로 440 (읍내동)  053-320-2111   
8                              대구광역시 남구 성당로 224 (대명동)  053-620-9119   
9                              대구광역시 서구 평리로 157 (중리동)  053-560-7288   
10                              대구광역시 동구 아양로 99 (신암동)  053-940-7119   
11                         대구광역시 달서구 달구벌대로 1632 (감삼동)  053-550-5019   
12                             대구광역시 남구 대명로 153 (대명동)  053-640-8575   
13               대구광역시 달서구 월배로 436, 지하1,지상1~11층 (송현동)  053-659-3119   
14                             대구광역시 남구 현충로 170 (대명동)  053-620-3177   
15                           대구광역시 달서구 감삼북길 141 (감삼동)  053-557-7119   
16  대구광역시 북구 호국로 807, 칠곡경북대학교병원(임상실습동 1~11층 포함) (학정동)  053-200-2100   
17                             대구광역시 달서구 월곡로 60 (도원동)  053-630-7014   

           위도          경도  응급실수  수술실수  가용응급실수  가용수술실수  ... 복부손상 사지접합 응급투석  \
0   35.840225  128.705733    10     4      10       4  ...    N    N    Y   
1   35.883009  128.663456    10     4      12       7  ...    N    N    N   
2   35.866235  128.604315    22    16       0      16  ...    Y    Y    Y   
3   35.853887  128.480130    28    24       0      22  ...    Y    Y    Y   
4   35.870693  128.588728    10     4       0       4  ...    N    N    Y   
5   35.814700  128.525095    11     5       0       5  ...    Y    N    Y   
6   35.843066  128.567426    24    18       4      17  ...    Y    Y    Y   
7   35.934880  128.549252    10     3       0       3  ...    N    N    N   
8   35.852967  128.574067     7     2       8       1  ...    Y    N    N   
9   35.859623  128.540561    10     4      16       4  ...    N    N    N   
10  35.883934  128.623835    28    13       0      11  ...    Y    N    Y   
11  35.852642  128.544969    10     8       8       7  ...    N    N    N   
12  35.839635  128.574355    10     3       7       3  ...    N    N    Y   
13  35.832748  128.553855    10     2       7       4  ...    N    N    Y   
14  35.847506  128.585131    30    19      10      26  ...    N    Y    N   
15  35.850823  128.541483    11     8       3       7  ...    N    N    Y   
16  35.956083  128.564408    17    13       6      19  ...    Y    N    N   
17  35.803137  128.551646    14     6       9       6  ...    N    N    N   

   조산산모 신생아 중증화상 응급실가용율 응급실포화도     거리    거리구분  
0     N   N    Y   1.00     원활  11.68  10km이상  
1     N   N    N   1.00     원활   7.57  10km이내  
2     Y   Y    Y   0.00     불가   2.11   5km이내  
3     N   Y    Y   0.00     불가   9.26  10km이내  
4     Y   N    Y   0.00     불가   0.71   2km이내  
5     Y   N    Y   0.00     불가   7.88  10km이내  
6     Y   N    Y   0.17     혼잡   3.15   5km이내  
7     N   N    Y   0.00     불가   7.86  10km이내  
8     N   N    N   1

In [772]:
# 중증질환자 수용이 가능하고, 응급실 수용이 가능한 병원 중 환자 위치에서 가까운 병원
# 조건1 : '중증화상' 중증질환자 수용 및 수술실 수용 가능한 응급으료기관 조회
# 조건2 :응급실 포화도가 불가가 아닌 병원
# 정렬 : 환자 위치를 중심으로 가장 가까운 병원부터 출력

patient= (35.8690760, 128.5811616)  # 환자 위치 : 서문시장 내 #Latitude, Longitude

# 조건1 : '중증화상' 중증질환자 수용 및 수술실 수용 가능한 응급으료기관 조회
special_m = '중증화상'
condition1 = (solution_df[special_m] == 'Y') & (solution_df['가용수술실수'] >= 1)

# 조건2 :응급실 포화도가 불가가 아닌 병원
condition2 = (solution_df['응급실포화도'] != '불가')

# 조건1, 2에 해당되는 응급의료기관 정보를 distance_df에 저장하기

distance_df =  solution_df[condition1 & condition2].copy()

# 환자와 병원간 거리를 저장할 빈 리스트 만들기 
# 변수명 : distance

distance = []

for idx, row in distance_df.iterrows():
    # haversine 함수를 이용하여 환자와 병원간 거리 구하기 
    # 단위 : km
    # 소수 둘째자리까지 구하기
    # distance 변수에 하나씩 채우기
    distance.append( round( haversine( patient, ( row['위도'], row['경도']) , unit = 'km'), 2)  )
    

#distance_df의 '거리' 컬럼에, distance 값 저장하기
distance_df['거리'] = distance #distance_df의 '거리' 컬럼에, distance 값 저장하기

# distancd_df의 '거리'에 따라 거리구분 분류 : cut()
# '거리구분' 명칭 : ['2km이내', '5km이내', '10km이내', '10km이상']

bins = [-1, 2, 5, 10, np.inf ] 
labels = ['2km이내', '5km이내', '10km이내', '10km이상']

distance_df['거리구분'] = pd.cut(distance_df['거리'], bins = bins, labels = labels )

# 환자 위치에서 가까운 거리순으로, 응급실 포화도가 원활한 순서로 정렬하기

distance_df.sort_values(['거리구분','응급실포화도'], ascending = [True, False])

병원코드                    병원명  \
9   A1300007                  대구의료원   
12  A1300034                 드림종합병원   
13  A1300045                   삼일병원   
14  A1300001                영남대학교병원   
6   A1300004             대구가톨릭대학교병원   
16  A1300010              칠곡경북대학교병원   
0   A1300081  (재)미리내천주성삼성직수도회천주성삼병원   

                                                   주소         응급연락처  \
9                              대구광역시 서구 평리로 157 (중리동)  053-560-7288   
12                             대구광역시 남구 대명로 153 (대명동)  053-640-8575   
13               대구광역시 달서구 월배로 436, 지하1,지상1~11층 (송현동)  053-659-3119   
14                             대구광역시 남구 현충로 170 (대명동)  053-620-3177   
6                          대구광역시 남구 두류공원로17길 33 (대명동)  053-650-3025   
16  대구광역시 북구 호국로 807, 칠곡경북대학교병원(임상실습동 1~11층 포함) (학정동)  053-200-2100   
0                  대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)  053-790-0555   

           위도          경도  응급실수  수술실수  가용응급실수  가용수술실수  ... 복부손상 사지접합 응급투석  \
9   35.859623  128.540561    10     4      16       4  ...    N    N    N   
12  35.839635  128.574355    10     3       7       3  ...    N    N    Y   
13  35.832748  128.553855    10     2       7       4  ...    N    N    Y   
14  35.847506  128.585131    30    19      10      26  ...    N    Y    N   
6   35.843066  128.567426    24    18       4      17  ...    Y    Y    Y   
16  35.956083  128.564408    17    13       6      19  ...    Y    N    N   
0   35.840225  128.705733    10     4      10       4  ...    N    N    Y   

   조산산모 신생아 중증화상 응급실가용율 응급실포화도     거리    거리구분  
9     N   N    Y   1.00     원활   3.81   5km이내  
12    Y   Y    Y   0.70     원활   3.33   5km이내  
13    Y   Y    Y   0.70     원활   4.73   5km이내  
14    Y   N    Y   0.33     보통   2.43   5km이내  
6     Y   N    Y   0.17     혼잡   3.15   5km이내  
16    Y   N    Y   0.35     보통   9.79  10km이내  
0     N   N    Y   1.00     원활  11.68  10km이상  

[7 rows x 23 columns]

### 9) 특정 중증질환 및 환자 위치에 따른 병원 실시간 정보 및 거리를 구하는 함수 작성하기

 * 작성 방법
    * 1) 함수선언하기
    * 2) 병원 리스트 csv 파일 불러오기
    * 3) 병원 실시간 정보 가져오기
    * 4) 병원 응급실 가용율, 포화도 계산하기
    * 5) 환자 수용 가능한 병원 구하기 
    * 6) 환자와 병원간 거리 구하기
    * 7) 거리 구간 구하기
    * 8) 결과값 반환하기

In [304]:
## tabulate library가 없는 경우 설치 필요
## pandas.to_markdown() 시 tabulate library가 필요함
!pip install tabulate 

In [773]:
import pandas as pd
from urllib.parse import quote
from haversine import haversine
import ssl
from urllib.request import urlopen
import numpy as np

context=ssl.create_default_context()
context.set_ciphers("DEFAULT")

#########################################
# 1) 함수 선언하기                       #
#########################################
# 함수명 : find_hospital
# 매개변수 : special_m (중증질환명), lati, long (환자 위치)

def find_hospital(special_m, lati, long ):
    #  [국립중앙의료원 - 전국응급의료기관 조회 서비스] 활용을 위한 개인 일반 인증키(Encoding) 저장

    key = "AsSDRMS8YsGyYCeyYZPWC%2BN6QSRKweQpU27iKc0HZeeSzPRlmk7jFERIjPJzH3%2FFwQc0UqtoPv1VYEXCiXFckA%3D%3D" 

    # city = 대구광역시, 인코딩 필요
    city = quote("대구광역시")

    ############################################################
    # 2) 병원 리스트 csv 파일 불러오기 (daegu_hospital_list.csv) #
    ############################################################
    solution_df = pd.read_csv('./daegu_hospital_list.csv')

    ############################################################
    # 3) 병원 실시간 정보 가져오기                               #
    ############################################################

    # 응급실 실시간 가용병상 

    url_realtime = "https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEmrrmRltmUsefulSckbdInfoInqire?serviceKey=" + key + "&STAGE1=" + city + "&numOfRows=100&pageNo=1"

    result = urlopen(url_realtime, context=context)
    emrRealtime_big = pd.read_xml(result, xpath='//item')

    ## 응급실 실시간 가용병상 정보에서 기관코드(hpid), 응급실 병상수('hvec'), 수술실 수('hvoc') 정보만 추출하여 emRealtime_small 변수에 저장
    ## emrRealtime_big 중 [hpid, hvec, hvoc] 컬럼 활용

    emrRealtime_small = emrRealtime_big[['hpid', 'hvec', 'hvoc']] ## emrRealtime_big 중 [hpid, hvec, hvoc] 컬럼 활용

    # solution_df와 emrRealtime_small 데이터프레임을 결합하여 solution_df에 저장
    solution_df =  pd.merge(solution_df, emrRealtime_small, on = 'hpid',how='inner')

    # 실시간 중증질환자 수용 가능 병원 조회

    url_acpt = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getSrsillDissAceptncPosblInfoInqire?serviceKey=' + key + "&STAGE1=" +city + "&numOfRows=100&pageNo=1"

    result = urlopen(url_acpt, context=context)
    emrAcpt_big =  pd.read_xml(result, xpath='.//item')

    ## 다른 API함수와 다르게 기관코드 컬럼명이 다름 (hpid --> dutyName)
    ## 기관코드 컬렴명을 'hpid'로 일치화시키기 위해, 컬럼명을 변경함

    emrAcpt_big.rename(columns = {'dutyName' : 'hpid'}, inplace = True)

    ## 실시간 중증질환자 수용 가능 병원정보에서 필요한 정보만 추출하여 emrAcpt_small 변수에 저장
    ## emrAcpt 중 [hpid, MKioskTy1, MKioskTy2, MKioskTy3, MKioskTy4, MKioskTy5, MKioskTy7,MKioskTy8, MKioskTy10, MKioskTy11] 컬럼 확인

    emrAcpt_small = emrAcpt_big[['hpid', 'MKioskTy1', 'MKioskTy2', 'MKioskTy3', 'MKioskTy4', 'MKioskTy5', 'MKioskTy7','MKioskTy8', 'MKioskTy10', 'MKioskTy11']]

    # solution_df와 emrAcpt_small 데이터프레임을 결합하여 solution_df에 저장
    solution_df = pd.merge(solution_df, emrAcpt_small, on = 'hpid',how = 'inner')

    ############################################################
    # 4) 자료 정비하기                                          #
    ############################################################

    # solution_df의 컬럼명 변경하기

    column_change = {  'hpid' : '병원코드',
                    'dutyName' : '병원명',
                    'dutyAddr' : '주소',
                    'dutyTel3' : '응급연락처',
                    'wgs84Lat' : '위도', 
                    'wgs84Lon' : '경도',
                    'hperyn' : '응급실수',
                    'hpopyn' : '수술실수',
                    'hvec' : '가용응급실수',
                    'hvoc' : '가용수술실수',
                    'MKioskTy1' : '뇌출혈', 
                    'MKioskTy2' : '뇌경색',
                    'MKioskTy3' : '심근경색',
                    'MKioskTy4' : '복부손상',
                    'MKioskTy5' : '사지접합',
                    'MKioskTy7' : '응급투석',
                    'MKioskTy8' : '조산산모',
                    'MKioskTy10' : '신생아',
                    'MKioskTy11' : '중증화상',                   
                    }

    solution_df = solution_df.rename(columns = column_change)

    # 중증질환 수용 가능 여부 데이터 중 정보 미제공, 불가능은 N로 변경 : replace

    solution_df = solution_df.replace("정보미제공","N")
    solution_df = solution_df.replace("불가능","N")

    ## 응급실수/가용응급실수, 수술실수/가용수술실 수가 0보다 작은 경우는 비정상 데이터로 추정
    ## 0보다 작은 수는 0으로 변경

    solution_df.loc[solution_df['응급실수']<0, '응급실수'] = 0
    solution_df.loc[solution_df['수술실수']<0, '수술실수'] = 0
    solution_df.loc[solution_df['가용응급실수']<0, '가용응급실수'] = 0
    solution_df.loc[solution_df['가용수술실수']<0, '가용수술실수'] = 0

    # 응급실 가용율을 구하여 새로운 컬럼으로 추가하기
    # 컬렴명 : '응급실가용율'
    # 산식 : 가용 응급실수 / 응급실 수
    # 소수 둘째 자리까지 구하기 round() 활용
    solution_df['응급실가용율'] = round(solution_df['가용응급실수'] / solution_df['응급실수'], 2)
    

    # 응급실 가용율이 1이 넘는 경우는 1로 대체
    solution_df.loc[solution_df['응급실가용율'] > 1, '응급실가용율'] = 1

    # 응급실 가용율에 따라 포화도 분류
    # 응급실 가용율 구분 단계 : ~0.1, 0.1 ~ 0.3, 0.3 ~ 0.6, 0.6 ~ 
    # 포화도 명칭 : ['불가', '혼잡', '보통', '원활']
    # pd.cut() 활용
    bins = [-np.inf, 0.1, 0.3, 0.6, np.inf]
    labels = ['불가', '혼잡', '보통', '원활']
    solution_df['응급실포화도'] = pd.cut(solution_df['응급실가용율'], bins=bins, labels=labels)

    
    ############################################################
    # 5) 환자 수용 가능한 병원 구하기                            #  
    ############################################################
    
    # 매개변수 special_m로 받은 중증질환이 중증질환 리스트에 포함될 경우
    # 중증질환 리스트 :  ['뇌출혈', '뇌경색', '심근경색', '복부손상', '사지접합', '응급투석', '조산산모', '신생아','중증화상' ]
        
    severe_diseases = ['뇌출혈', '뇌경색', '심근경색', '복부손상', '사지접합', '응급투석', '조산산모', '신생아','중증화상']

    if special_m in severe_diseases:
        # 조건1 : special_m 중증질환자 수용이 가능하고
        # 조건2 : 응급실 포화도가 불가가 아닌 병원
        condition1 = (solution_df[special_m] == 'Y') & (solution_df['가용수술실수']>=1)
        condition2 = (solution_df['응급실포화도'] != '불가')
        
        # 조건1, 2에 해당되는 응급의료기관 정보를 distance_df에 저장하기
        distance_df = solution_df[condition1 & condition2].copy()
    
    # 매개변수 special_m 값이 중증질환 리스트에 포함이 안되는 경우        
    else:
        # 조건1 : 응급실 포화도가 불가가 아닌 병원
        condition1 = solution_df['응급실포화도'] != '불가'
        
        # 조건1에 해당되는 응급의료기관 정보를 distance_df에 저장하기
        distance_df = solution_df[condition1].copy()


    ############################################################
    # 6) 환자와 병원간 거리 구하기                               # 
    ############################################################
    distance = []
    patient = (lati, long)

    for idx, row in distance_df.iterrows():
        distance.append(round( haversine(patient,(row['위도'], row['경도']), unit='km') , 2))

    distance_df.loc[:,'거리'] = distance.copy()

    
    
    
    ############################################################
    # 7) 거리 구간 구하기                                       #
    ############################################################
    bins = [-np.inf, 2, 5, 10, np.inf]
    labels = ['2km이내', '5km이내', '10km이내', '10km이상']
    
    distance_df.loc[:, '거리구분'] = pd.cut(distance_df['거리'], bins=bins, labels=labels)

    ############################################################
    # 8) 결과값 반환하기                                        #
    ############################################################

    return distance_df


In [774]:
# 중증환자 수용 가능한, 가까운 병원 조회 #1
# 환자 정보 
#   - 중증 질환 : 응급투석
#   - 환자 위치 : 대구역 근처 (35.8765167, 128.5972922))
# 거리순, 응급실 포화도 순으로 결과 출력하기 (pandas의 sort_values(), to_markdown() 활용)
print(find_hospital('응급투석', 35.8765167, 128.5972922).sort_values(['거리구분', '응급실포화도','거리'], ascending=[True, False, True]).to_markdown())

|    | 병원코드   | 병원명                                   | 주소                                                     | 응급연락처   |    위도 |    경도 |   응급실수 |   수술실수 |   가용응급실수 |   가용수술실수 | 뇌출혈   | 뇌경색   | 심근경색   | 복부손상   | 사지접합   | 응급투석   | 조산산모   | 신생아   | 중증화상   |   응급실가용율 | 응급실포화도   |   거리 | 거리구분   |
|---:|:-----------|:-----------------------------------------|:---------------------------------------------------------|:-------------|--------:|--------:|-----------:|-----------:|---------------:|---------------:|:---------|:---------|:-----------|:-----------|:-----------|:-----------|:-----------|:---------|:-----------|---------------:|:---------------|-------:|:-----------|
| 12 | A1300034   | 드림종합병원                             | 대구광역시 남구 대명로 153 (대명동)                      | 053-640-8575 | 35.8396 | 128.574 |         10 |          3 |              7 |              3 | N        | N        | N          | N          | N          | Y          | Y          | Y        | Y          |           0.

In [589]:
# 중증환자 수용 가능한, 가까운 병원 조회 #2
# 환자 정보 
#   - 중증 질환 : 심근경색
#   - 환자 위치 : 수성못 근처 (35.8259090, 128.616528))
# 거리순, 응급실 포화도 순으로 결과 출력하기 (pandas의 sort_values(), to_markdown() 활용)

print(find_hospital('심근경색', 35.8765167, 128.5972922).sort_values(['거리구분', '응급실포화도', '거리'], ascending=[True, False, True]).to_markdown())

|    | 병원코드   | 병원명         | 주소                                             | 응급연락처   |    위도 |    경도 |   응급실수 |   수술실수 |   가용응급실수 |   가용수술실수 | 뇌출혈   | 뇌경색   | 심근경색   | 복부손상   | 사지접합   | 응급투석   | 조산산모   | 신생아   | 중증화상   |   응급실가용율 | 응급실포화도   |   거리 | 거리구분   |
|---:|:-----------|:---------------|:-------------------------------------------------|:-------------|--------:|--------:|-----------:|-----------:|---------------:|---------------:|:---------|:---------|:-----------|:-----------|:-----------|:-----------|:-----------|:---------|:-----------|---------------:|:---------------|-------:|:-----------|
|  8 | A1300076   | 대구굿모닝병원 | 대구광역시 남구 성당로 224 (대명동)              | 053-620-9119 | 35.853  | 128.574 |          7 |          2 |              8 |              1 | N        | Y        | Y          | Y          | N          | N          | N          | N        | N          |           1    | 원활           |   3.35 | 5km이내    |
|  5 | A1300038   | 나사렛종합병원 | 대구광역시 달서구 월배로 97, 나사렛병원 (진천동) | 0

## 미션#1 Clear
## 수고하셨습니다!!

In [583]:
import requests
from urllib.parse import quote

# 네이버 지도 API 사용을 위한 헤더 정보
headers = {
    'X-NCP-APIGW-API-KEY-ID': 'o58xyfyhpo',
    'X-NCP-APIGW-API-KEY': 'AnY8Z5u5yo69Bh4vPuNLmiSP011OiFR8nHOYsK7V'
}

# 특정 위치 검색 (예: 서문시장)
def get_lat_lng(location):
    url = f"https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={quote(location)}"
    response = requests.get(url, headers=headers)
    print(response.text)
    if response.status_code == 200:
        data = response.json()
        if 'addresses' in data and len(data['addresses']) > 0:
            lat = data['addresses'][0]['y']
            lng = data['addresses'][0]['x']
            return (float(lat), float(lng))
    return None

# 예제
location_name = "서울특별시 중구 명동길"
patient = get_lat_lng(location_name)
print(patient)

{"error":{"errorCode":"230","message":"Forbidden","details":""}}
None
